In [2]:
import pandas as pd
import numpy as np

### Практики для разбора

In [ ]:
#### Введем колонку, нумирующие последовательные сообщения от одного и того же sender_type

def sequence(x):
    """Функция для подсчета длинны последовательности входящих значений"""
    if x == sequence.val:
        sequence.count += 1
    else:
        sequence.val = x
        sequence.count = 1
    
    return sequence.count
    
sequence.val = ''
sequence.count = 1

data['seq_sender_type'] = data['sender_type'].apply(sequence)

# Проверка

data[data['chat_id'] == chat_ids.pop(0)][['chat_id', 'sender_type', 'seq_sender_type', 'message']]

In [ ]:
метод cumsum()

# cumcount
user_send_data['message_order'] = user_send_data[['chat_id', 'message']].groupby('chat_id').cumcount()

In [ ]:
# добавление chat_id
data['chat_id'] = data['date'].astype('str') + data['history_id'].astype('str')

In [ ]:
# groupby + transform
data['flag_answered'] = data.groupby('history_id')['sender_type'].transform('nunique')

In [ ]:
# map
data['flag_answered'] = (data['flag_answered'] != 1).map(int)

In [ ]:
# groupby

data['msg_diff'] = (
    data.sort_values(by=['datetime'], kind='stable')
    .groupby('history_id', as_index = False)['datetime']
    .diff()
    .datetime.dt.total_seconds()
    .fillna(0)
    .astype(int)
)

In [ ]:
# .grouper.group_info[0]+1
data['chat_id'] = data.groupby(['history_id','date']).grouper.group_info[0]+1

# ?
# np.logical_and(), cumcount
data.loc[np.logical_and(data.groupby('history_id').history_id.cumcount()==0, data.sender_type == 'USER'), 'is_init'] = 1

In [ ]:
# ?
data['start_chat'] = data['is_init'] | data['have_greetings']

In [ ]:
# создадим новый столбец, равный разнице во времени между этим сообщением и следующим
data['next_message_time_duration'] = data['datetime'].diff().shift(-1)
data['next_message_time_duration'] = abs(data['next_message_time_duration'])

In [ ]:






# найдем только те строки, в которых оператор прощается с клиентом
next_message_range = data.loc[(data.message.str.endswith('Всего Вам доброго!')) \
                              | (data.message.str.startswith('Спасибо Вам за обращение'))].copy()

bins = [
    pd.Timedelta(hours = 0),
    pd.Timedelta(hours = 2),
    pd.Timedelta(hours = 4),
    pd.Timedelta(hours = 6),
    pd.Timedelta(hours = 12),
    pd.Timedelta(hours = 18),
    pd.Timedelta(hours = 23),
    pd.Timedelta(days = 2),
    pd.Timedelta(days = 100)
]

labels = ['0-2hours', '2-4hours', '4-6hours', '6-12hours', '12-18hours', '18-23hours', '1-2 days', '2+ days']

# посмотрим как ранжирована разница между прощанием оператора с клиентом и слудющим сообщением
next_message_range['bins'] = pd.cut(next_message_range['next_message_time_duration'], bins, labels = labels)

(
    next_message_range
                            .groupby('bins')['next_message_time_duration']
                            .agg('count')
                            .reset_index()
                            .sort_values(by='next_message_time_duration', ascending=False)
)

In [ ]:
# Менеджмент памяти

del next_message_range
gc.collect()

#### Как убрать дубликаты из списка, при этом сохранив порядок?

In [3]:
items = [1,1, 2,2, 3,3, 4,4, 5,5]
list(dict.fromkeys(items))

[1, 2, 3, 4, 5]

#### Как распаковать список списков в один список сохранением порядка?

In [6]:
list_of_lists = [[1,1], ['2','2'], ['three', 'three']]
sum(list_of_lists, [])

[1, 1, '2', '2', 'three', 'three']

### Как сопоставить несколько dataframe-ов и посчитать от них среднее?

In [2]:
df1 = pd.DataFrame({
    'val1': [1,1,1,1],
    'val2': [10,10,10,10]
})

df2 = pd.DataFrame({
    'val1': [2,2,2,2],
    'val2': [20,20,20,20]
})

df3 = pd.DataFrame({
    'val1': [3,3,3,3],
    'val2': [30,30,30,30]
})

df_concat = pd.concat([df1, df2, df3])
df = df_concat.groupby(df_concat.index).mean()
df

,val1,val2
0,2.0,20.0
1,2.0,20.0
2,2.0,20.0
3,2.0,20.0


### Как разом изменить расположение группы колонок?

In [12]:
df = pd.DataFrame({
    'col1': ['A','B','C'],
    'g1': [1,2,3], 
    'g2': [3,2,1], 
    'g3': [5,6,7],
    'g4': [7,6,5], 
    'gn': [90,80,70], 
    'a1': ['1a','1a','1a'], 
    'a2': ['2a','2a','2a'], 
    'a3': ['3a','3a','3a']
})

df

,col1,g1,g2,g3,g4,gn,a1,a2,a3
0,A,1,3,5,7,90,1a,2a,3a
1,B,2,2,6,6,80,1a,2a,3a
2,C,3,1,7,5,70,1a,2a,3a


#### Способ 1 - ввести новую последовательность целиком

In [13]:
df1 = df[['col1', 'g1', 'g2', 'g3', 'g4', 'gn', 'a1', 'a2', 'a3']]
df1

,col1,g1,g2,g3,g4,gn,a1,a2,a3
0,A,1,3,5,7,90,1a,2a,3a
1,B,2,2,6,6,80,1a,2a,3a
2,C,3,1,7,5,70,1a,2a,3a


##### А что если колонок будет 100 или больше?

#### Способ 2 - спосмощью индексации колонок?

#### Способ 3 - спомощью списка колонок и *

In [14]:
g = [col for col in df.columns if 'g' in col]
a = [col for col in df.columns if 'a' in col]
print(g, a)
df2 = df[['col1', *a, *g]]
df2

['g1', 'g2', 'g3', 'g4', 'gn'] ['a1', 'a2', 'a3']


,col1,a1,a2,a3,g1,g2,g3,g4,gn
0,A,1a,2a,3a,1,3,5,7,90
1,B,1a,2a,3a,2,2,6,6,80
2,C,1a,2a,3a,3,1,7,5,70


### Как добавить несколько колонок одной функцией?
https://stackoverflow.com/questions/39050539/how-to-add-multiple-columns-to-pandas-dataframe-in-one-assignment

In [7]:
def ex(row):
    a = row.a
    b = row.b
    
    c = a + b
    d = a - b
    e = a > b
    
    return c, d, e

In [8]:
df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [5, 6, 7]
})
df.head()

,a,b
0,1,5
1,2,6
2,3,7


#### Способ 1 - списки + метод iterrows()

In [15]:
c_list = list()
d_list = list()
e_list = list()

for _, row in df.iterrows():
    c, d, e = ex(row)
    
    c_list.append(c)
    d_list.append(d)
    e_list.append(e)
    
df['c'] = c_list
df['d'] = d_list
df['e'] = e_list

df

,a,b,c,d,e
0,1,5,6,-4,False
1,2,6,8,-4,False
2,3,7,10,-4,False


#### Способ 2 - apply + apply

In [16]:
df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [5, 6, 7]
})

df[['c','d','e']] = df[['a','b']].apply(ex, axis=1).apply(pd.Series)
df

,a,b,c,d,e
0,1,5,6,-4,False
1,2,6,8,-4,False
2,3,7,10,-4,False


### Как сменить или сбросить header?

In [13]:
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [14]:
# сбросить header
df.columns = range(df.shape[1])

# определить новый header
df.columns = new_header

df

,a,b,c
0,1,4,7
1,2,5,8
2,3,6,9
